AIT-664, DL3

Keith Maly

Week 10 Assignment: Hands-on-2: Regression Models

Due: November 3, 2024

1. Environment Setup
- System UUID
- Mount Drive and Wrap text w/in Google Collab
- Library Import

In [ ]:
# capture my UUID within Google Colab
import uuid
my_name = "Keith Maly"
print (my_name + ": " + str(uuid.uuid1()))

Keith Maly: 06a400c6-9701-11ef-a8fb-0242ac1c000c


In [ ]:
# mount drive, set working directory and wrap text

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/AIT-664/hands-on-2'
#!ls

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/AIT-664/hands-on-2


In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
!pip install prettytable
from prettytable import PrettyTable

2. Import Data, Null Values removal

- Note: I am using the Boston Housing Set, which is all numeric values, no categorical per the Kaggle Site
- Assign Target and Response Variables
- Test Train Split



In [ ]:
# import data
bost_df = pd.read_csv('Boston-house-price-data.csv')
print('Before null removal', bost_df.shape)

# remove any nulls
bost_df.dropna(inplace=True)
print('After null removal', bost_df.shape)

Before null removal (506, 14)
After null removal (506, 14)


In [ ]:
bost_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [ ]:
# assign target and response variables
X = bost_df[['CRIM','ZN','INDUS','CHAS','NOX','RM',
             'AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']]
y = bost_df['MEDV']

# Scale the features for better performance of regression models
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

3. Begin Regression Modeling
- multivariate linear regression
- random forest (default settings)
- random forest (more specific settings)
- cross-validation
- gradient boost
- ridge
- lasso
- elastic

Measures for comparision: RMSE, R^2

In [ ]:
# Initialize and train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_linear = linear_model.predict(X_test)

# Evaluate the Linear Regression model
LR_MSE = round(mean_squared_error(y_test, y_pred_linear),3)
LR_R2 = round(r2_score(y_test, y_pred_linear), 3)
print("Linear Regression Mean Squared Error:", LR_MSE)
print("Linear Regression R^2 Score:", LR_R2 )

Linear Regression Mean Squared Error: 21.517
Linear Regression R^2 Score: 0.711


In [ ]:
# Initialize and train the Random Forest Regressor (default)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_rf = rf_model.predict(X_test)

RF_def_MSE = round(mean_squared_error(y_test, y_pred_rf),3)
RF_def_R2 = round(r2_score(y_test, y_pred_rf),3)

# Evaluate the Random Forest Regressor
print("Random Forest Regressor Mean Squared Error:", RF_def_MSE )
print("Random Forest Regressor R^2 Score:", RF_def_R2)

Random Forest Regressor Mean Squared Error: 9.61
Random Forest Regressor R^2 Score: 0.871


In [ ]:
# Create a RandomForestRegressor with regularization parameters
regressor = RandomForestRegressor(
    n_estimators=100,          # Number of trees
    max_depth=10,              # Limit the depth of each tree
    min_samples_split=5,       # Minimum samples required to split an internal node
    min_samples_leaf=4,        # Minimum samples required to be at a leaf node
    max_features='sqrt',       # Use the square root of the total features at each split
    random_state=42
)

# Fit the model
regressor.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = regressor.predict(X_test)
RF_reg_MSE = round(mean_squared_error(y_test, y_pred),3)
RF_reg_R2 = round(r2_score(y_test, y_pred_rf),3)
print("Random Forest Regressor Mean Squared Error:", RF_reg_MSE)
print("Random Forest Regressor R^2 Score:", RF_reg_R2)

Random Forest Regressor Mean Squared Error: 12.894
Random Forest Regressor R^2 Score: 0.871


In [ ]:
# Cross Validation
# Define the model with regularization parameters
regressor = RandomForestRegressor(
    n_estimators=300,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=4,
    max_features='sqrt',
    random_state=42
)

# Set up k-fold cross-validation (5 folds)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate MSE for each fold
scores = cross_val_score(
    regressor, X_scaled, y, cv=kf, scoring=make_scorer(mean_squared_error)
)

# Calculate the mean and standard deviation of the MSE scores
CV_mean_mse = round(np.mean(scores),3)
std_mse = np.std(scores)
CV_R2 = round(r2_score(y_test, y_pred),3)

print(f"Mean MSE from cross-validation: {CV_mean_mse:.3f}")
print(f"R^2 Score: {CV_R2:.3f}")

Mean MSE from cross-validation: 15.355
R^2 Score: 0.827


In [ ]:
# gradient boost method
# define the gradient boosting model with different parameters
gbm_model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=3, random_state=42)
gbm_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_gbm = gbm_model.predict(X_test)

gbm_mse = round(mean_squared_error(y_test, y_pred_gbm), 3)
gbm_R2 = round(r2_score(y_test, y_pred_gbm), 3)

# Evaluate the Gradient Boosting Regressor
print("Gradient Boosting Regressor Mean Squared Error:", gbm_mse )
print("Gradient Boosting Regressor R^2 Score:", gbm_R2)

Gradient Boosting Regressor Mean Squared Error: 7.757
Gradient Boosting Regressor R^2 Score: 0.896


In [ ]:
# L2 Regularization: Ridge Regression
ridge = Ridge(alpha=1.0)  # alpha controls the regularization strength; higher means more regularization
ridge.fit(X_train, y_train)
ridge_predictions = ridge.predict(X_test)

ridge_mse = round(mean_squared_error(y_test, ridge_predictions),3)
ridge_R2 = round(r2_score(y_test, ridge_predictions),3)

print("Ridge MSE:", ridge_mse)
print("Ridge R^2 Score:", ridge_R2)

# L1 Regularization: Lasso Regression
lasso = Lasso(alpha=0.1)  # alpha is the regularization parameter
lasso.fit(X_train, y_train)
lasso_predictions = lasso.predict(X_test)

lasso_mse = round(mean_squared_error(y_test, lasso_predictions),3)
lasso_R2 = round(r2_score(y_test, lasso_predictions),3)
print("Lasso MSE:", lasso_mse)
print("Lasso R^2 Score:", lasso_R2)

# L1 + L2 Regularization: Elastic Net
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)  # l1_ratio balances between L1 and L2 (0 = pure L2, 1 = pure L1)
elastic_net.fit(X_train, y_train)
elastic_net_predictions = elastic_net.predict(X_test)

elastic_mse = round(mean_squared_error(y_test, elastic_net_predictions),3)
elastic_R2 = round(r2_score(y_test, elastic_net_predictions), 3)
print("Elastic Net MSE:", elastic_mse)
print("Elastic Net R^2 Score:", elastic_R2)

Ridge MSE: 21.549
Ridge R^2 Score: 0.711
Lasso MSE: 22.798
Lasso R^2 Score: 0.694
Elastic Net MSE: 22.48
Elastic Net R^2 Score: 0.698


4. Summary Statistics and Analysis

In [ ]:

# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Regression Method", "MSE", "R^2"])

# Add rows
myTable.add_row(["Linear Regression", LR_MSE, LR_R2])
myTable.add_row(["RF(def)", RF_def_MSE, RF_def_R2])
myTable.add_row(["RF(reg)", RF_reg_MSE, RF_reg_R2])
myTable.add_row(["Cross-Validation", CV_mean_mse, CV_R2])
myTable.add_row(["Gradient Boost", gbm_mse, gbm_R2])
myTable.add_row(["Ridge", ridge_mse, ridge_R2])
myTable.add_row(["Lasso", lasso_mse, lasso_R2])
myTable.add_row(["Elastic", elastic_mse, elastic_R2])

print('Evaluation Summary')
print(myTable)

Evaluation Summary
+-------------------+--------+-------+
| Regression Method |  MSE   |  R^2  |
+-------------------+--------+-------+
| Linear Regression | 21.517 | 0.711 |
|      RF(def)      |  9.61  | 0.871 |
|      RF(reg)      | 12.894 | 0.871 |
|  Cross-Validation | 15.355 | 0.827 |
|   Gradient Boost  | 7.757  | 0.896 |
|       Ridge       | 21.549 | 0.711 |
|       Lasso       | 22.798 | 0.694 |
|      Elastic      | 22.48  | 0.698 |
+-------------------+--------+-------+


Analysis: Overall, all of the models performed well, with all models accounting for at least 69% of variance. Overall, the gradient boost model performed the best as measured by the lowest MSE and best R^2.  Lasso, performed most poorly (in relative terms, with the highest MSE and lowest R^2. In terms of compute, cross-validation seemed to take the longest, but all computed near instanteously on the most basic runtime type in Google Colab

References
- Hashimi, I, Income_regression_models.ipynb file, from AIT-664 BB Site
- Geeks for Geeks, **How to make a table in Python?**, 2022-02-22,https://www.geeksforgeeks.org/how-to-make-a-table-in-python/